# 국토교통부_상업업무용 부동산 매매 신고 조회 서비스


- 데이터 출처: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057267  
- 지역코드 출처: 주민등록주소코드 변경내역(2018. 1. 22. 현재)    https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=61552 내 첨부파일 jscode20180122.zip   
주의: 일일 트래픽 1000회 제한

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

## 함수 설정

In [19]:
def collect_nrg_trade(ym, lawd_cd) :
    
    # 제공받은 API 
    API_KEY = 'bHg%2FpafJkgCqoGEFC3cI5IvynqkFuwK53BYhBu3bqMz7I%2BX%2BEYH8R9jV%2FCMc06dm11DO%2FksvWzs5sIUeyCjiTQ%3D%3D'
    
    # URL
    url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'
 
    # 요청메시지
    url = url + '?serviceKey=' + API_KEY + '&LAWD_CD=' + lawd_cd + '&DEAL_YMD=' + ym 
 
    resultXML = urlopen(url)
    result = resultXML.read()
    xmlsoup = BeautifulSoup(result, 'lxml-xml')
 
    te = xmlsoup.findAll('item')
 
    # 데이터프레임 결과물
    nrg_trade = pd.DataFrame()
 
    for t in te:
        deal_amt = t.find('거래금액').text

        try:
            build_y = t.find('건축년도').text
        except:
            build_y = "" 

        deal_y = t.find('년').text
        deal_m = t.find('월').text
        deal_day = t.find('일').text
        area_build = t.find('건물면적').text
        build_use = t.find('건물주용도').text
        
        try:
            class_deal = t.find('구분').text
        except:
            class_deal = "" 

        dong = t.find('법정동').text
        area_plot = t.find('대지면적').text
        
        try:
            land_use = t.find('용도지역').text
        except:
            land_use = "" 
        
        region_code = t.find('지역코드').text
        try:
            floor = t.find('층').text        
        except:
            floor = ""  

 
        temp = pd.DataFrame(([[deal_amt, build_y, deal_y, deal_m, deal_day, 
                               area_build, build_use, class_deal, dong, area_plot, 
                               land_use, region_code, floor]]),
                           columns = ['deal_amt', 'build_y', 'deal_y', 'deal_m', 'deal_day', 
                                      'area_build', 'build_use', 'class_deal', 'dong', 
                                      'area_plot', 'land_use', 'region_code', 'floor'])
        nrg_trade = pd.concat([nrg_trade, temp])
 
    nrg_trade = nrg_trade.reset_index(drop=True)
 
    return nrg_trade

## 수집

#### 자료 수집 현황
2020년 data => 7/19 완료    

### **** 2019년부터 진행 예정 ****

In [20]:
if __name__ == "__main__":

    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"]=="서울특별시") | (code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    nrg_trade = pd.DataFrame()

    # 기간설정(연월: 'yyyymm')
    ym = list(['202006', 
               '202007', '202008', '202009', '202010', '202011','202012'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_nrg_trade(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            nrg_trade = pd.concat([nrg_trade, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

11110, 63개 완료
11140, 134개 완료
11170, 80개 완료
11200, 39개 완료
11215, 49개 완료
11230, 96개 완료
11260, 19개 완료
11290, 33개 완료
11305, 45개 완료
11320, 33개 완료
11350, 30개 완료
11380, 44개 완료
11410, 27개 완료
11440, 68개 완료
11470, 42개 완료
11500, 58개 완료
11530, 75개 완료
11545, 28개 완료
11560, 355개 완료
11590, 22개 완료
11620, 32개 완료
11650, 119개 완료
11680, 211개 완료
11710, 67개 완료
11740, 46개 완료
41110, 0개 완료
41111, 30개 완료
41113, 40개 완료
41115, 29개 완료
41117, 49개 완료
41130, 0개 완료
41131, 38개 완료
41133, 68개 완료
41135, 89개 완료
41150, 20개 완료
41170, 0개 완료
41171, 20개 완료
41173, 53개 완료
41190, 155개 완료
41210, 54개 완료
41220, 48개 완료
41250, 18개 완료
41270, 0개 완료
41271, 43개 완료
41273, 101개 완료
41280, 0개 완료
41281, 89개 완료
41285, 60개 완료
41287, 67개 완료
41290, 6개 완료
41310, 19개 완료
41360, 87개 완료
41370, 36개 완료
41390, 117개 완료
41410, 32개 완료
41430, 18개 완료
41450, 51개 완료
41460, 0개 완료
41461, 32개 완료
41463, 80개 완료
41465, 45개 완료
41480, 103개 완료
41500, 21개 완료
41550, 39개 완료
41570, 84개 완료
41590, 131개 완료
41610, 47개 완료
41630, 24개 완료
41650, 32개 완료
41670, 10개 완료
41800, 6개 완료
41820

In [21]:
nrg_trade

,deal_amt,build_y,deal_y,deal_m,deal_day,area_build,build_use,class_deal,dong,area_plot,land_use,region_code,floor,sido,sigungu
0,"400,000",1992,2020,6,21,246.85,제1종근린생활,,통의동,246.6,제2종일반주거,11110,,서울특별시,종로구
1,"235,000",,2020,6,12,59.5,제1종근린생활,,적선동,122.3,제2종일반주거,11110,,서울특별시,종로구
2,"33,250",1986,2020,6,30,80.25,제1종근린생활,,적선동,,일반상업,11110,,서울특별시,종로구
3,"65,000",2008,2020,6,15,66.15,제2종근린생활,,사직동,,준주거,11110,,서울특별시,종로구
4,"53,200",1985,2020,6,19,68.47,업무,,당주동,,일반상업,11110,,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,"49,000",2011,2020,12,31,113.92,기타,,서종면 수입리,636,보전관리,41830,,경기도,양평군
25,"22,692",2000,2020,12,11,66.6,제2종근린생활,,단월면 부안리,600,생산관리,41830,,경기도,양평군
26,"62,726",1999,2020,12,15,561.64,숙박,,개군면 하자포리,1053,보전관리,41830,,경기도,양평군
27,"18,000",2019,2020,12,30,66.08,제1종근린생활,,개군면 하자포리,,일반상업,41830,1,경기도,양평군


In [22]:
nrg_trade2

,Unnamed: 0,deal_amt,build_y,deal_y,deal_m,deal_day,area_build,build_use,class_deal,dong,area_plot,land_use,region_code,floor,sido,sigungu
0,0,"80,000",1962.0,2020,1,10,50.99,제1종근린생활,NaN,당주동,13.55,일반상업,11110,NaN,서울특별시,종로구
1,1,"80,000",1962.0,2020,1,10,50.99,제1종근린생활,NaN,당주동,13.55,일반상업,11110,NaN,서울특별시,종로구
2,2,"80,000",1962.0,2020,1,10,50.99,제1종근린생활,NaN,당주동,13.55,일반상업,11110,NaN,서울특별시,종로구
3,3,"80,000",1962.0,2020,1,10,50.99,제1종근린생활,NaN,당주동,13.55,일반상업,11110,NaN,서울특별시,종로구
4,4,"80,000",1962.0,2020,1,10,50.99,제1종근린생활,NaN,당주동,13.55,일반상업,11110,NaN,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14487,1,"14,000",2017.0,2020,5,4,19.84,제2종근린생활,NaN,양평읍 공흥리,173,자연녹지,41830,NaN,경기도,양평군
14488,2,"111,664",1997.0,2020,5,28,979.50,제2종근린생활,NaN,양서면 양수리,420,준주거,41830,NaN,경기도,양평군
14489,3,"46,560",2019.0,2020,5,27,154.55,제2종근린생활,NaN,양서면 증동리,505,보전관리,41830,NaN,경기도,양평군
14490,4,750,1988.0,2020,5,8,4.50,숙박,NaN,옥천면 신복리,,계획관리,41830,NaN,경기도,양평군


In [23]:
nrg_trade = pd.concat([nrg_trade2, nrg_trade])

In [24]:
nrg_trade.to_csv('nrg_trade_2020.csv', index=False)